In [ ]:
단어 설명
    영역 : 주정통의 10가지 분야 ex) unix, windows server, ....
    호스트 : 각 영역에 해당하는 장치 ex) server1, router1, switch1
    점검항목 : 각 영역별 점검해야 하는 항목 ex) U_01, W_01

단어간 관계
    영역 1 - 호스트 n
    호스트 1 - 점검항목 n

메인 코드 설명
    
    
    반복되는 여러개의 task 중 하나에서 영역에 해당하는 호스트들을 for문으로 반복하되, 병렬실행한다.
    반복되는 여러개의 task 중 하나에서 영역에 해당하는 점검항목들을 for문으로 반복하되, 병렬실행한다.
    
메인 코드 흐름
    1. db에서 호스트 정보를 읽어온다. 
        : db에서 호스트 정보를 받아오면 각 영역별로 구분해서 리스트를 만든다.
    2. 각 영역별로 호스트 리스트를 만든다.
        : 여러 개의 영역들을 for문으로 반복하되, 병령실행한다. 
    3. threadpoolexcutor로 영역을 병렬실행한다. -단 max-worker = 1
    4. threadpoolexcutor로 영역에 해당하는 호스트들을 병렬실행한다. max-worker = 5
    5. threadpoolexcutor로 호스트마다 영역에 해당하는 점검함수들을 병렬실행한다.  max-worker = 10

서브 클래스 설명
    메인 클래스에 의해서 호출되면 호스트 장비에 ssh로 접속해서 특정 명령어를 날리고 
서브 클래스 코드
    1. host 정보를 받아서 paramiko/netmiko로 특정 명령어를 전송한다.
    2. 결과값에서 필요한 정보를 뽑아낸다.
    3. 정보값을 바탕으로 지키고 있는지 안 지키고 있는지 판단한다.
    4. 정보 생성 - 날짜, 호스트, 영역, 점검코드, 결과/ 점수(중요도)
    5. 정보 db에 저장



병렬실행 시 동시최대 수
#threadpoolexcutor로 전체 영역 실행
threadpoolexcutor로 전체 호스트실행
threadpoolexcutor로 전체 점검사항 체크


In [ ]:
!pip install pymysql
!pip install paramiko
!pip install netmiko

In [25]:
import pymysql
import os
import paramiko
import netmiko
from concurrent.futures import ThreadPoolExecutor, as_completed

class main_func:

    def query(cmd):
        # db.txt 파일에서 정보 읽기
        with open("db.txt", "r") as f:
            lines = [line.strip() for line in f.readlines()]
            ip, port, id, pw = lines[0], lines[1], lines[2], lines[3]
    
        # DB 연결
        db = pymysql.connect(host=ip, port=int(port), user=id, passwd=pw, charset='utf8', db="주정통")
        cur = db.cursor()
        cur.execute(cmd)
        rows = []
        if "SELECT" in cmd or "SHOW" in cmd:
            rows = cur.fetchall()
        elif "INSERT" in cmd:
            db.commit()
            rows = ["query ok"]
        db.close()
        return list(rows)

    def gethost(case) :
        
        for category in categories:
            cmd = "SELECT * FROM hosts WHEN category =  " + category
            rows = query(cmd)
            for i in rows:
                #print(i)
                host = {}
                host["category"] = i[1]
                host["hostname"] = i[2]
                host["ip"] = i[3]
                host["username"] = i[4]
                host["password"] = i[5]
                if len(i) == 6 :
                    host["os"] = i[4]
                ahost_list.append(host.copy())
        
    
    def para_connect(host,cmd,time1) :
        cli = paramiko.SSHClient() # ssh 클라이언트 인스턴스를 생성 ---> cli 객체
        cli.set_missing_host_key_policy(paramiko.AutoAddPolicy)  # 접속할 때 에러 메시지 처리 
        cli.connect(host["ip"],username=host["username"],password=host["password"]) #ip,username,password로 ssh 연결
        connection = cli.invoke_shell()

        result = ""
        cmd_list = cmd.strip().split("\n")
        for cmd_item in cmd_list:
            connection.send(cmd_item + "\n")
            time.sleep(time1)
            result += connection.recv(65535).decode('utf-8')
            result += "\n"
        cli.close()
        
        return result
    
    #6. 명령어 전달 및 결과값 저장 def net_connect(user) return result
    def net_connect (host, cmd,time1):
        net_connect = netmiko.ConnectHandler(device_type="cisco_ios", ip=host["ip"],username=host["username"],password=host["password"],timeout=15) # ssh 연결
        net_connect.enable() #관리자 모드 실행
        cmdt = cmd.strip()
        tmp = cmd.split("\n")
        if len(tmp) == 1 :
            result =net_connect.send_command(tmp[0])
        else :
            result = net_connect.send_config_set(tmp)
        time.sleep(time1)
        net_connect.disconnect()
    
        return result 

    #def write_inventory()

    #def run_ansible()

    #def refine_data()

    #def insert_data()


    
#test 코드

#result = main.query("router","SELECT * FROM host")
#print(result)
main_func.gethost()


#main 실행 코드


TypeError: main_func.gethost() missing 1 required positional argument: 'case'

In [ ]:
class unix

In [22]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 예제로 사용할 작업 함수
def task(n):
    print(f"작업 {n} 시작")
    time.sleep(1)  # 1초 동안 대기 (실제 작업처럼 보이도록)
    print(f"작업 {n} 완료")
    return f"결과 {n}"
def print1():
    print("1")
def print2():
    print("2")
def print3():
    print("3")
def print4():
    print("4")
def print5():
    print("5")
# 병렬 실행
def main1():
    nums = [1, 2, 3, 4, 5]
    results = []
    # 최대 3개의 스레드를 사용
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(task, n) for n in nums]

        for future in as_completed(futures):
            #print(">>", future.result())
            results.append(future.result())
    return results
def main2(result):
    print("main2 start")
    for a in result:
        print(a)

def main3():
    tasks = [print1,print2,print3,print4,print5] 
    nums = [1, 2, 3, 4, 5]
    results = []
    # 최대 3개의 스레드를 사용
    with ThreadPoolExecutor(max_workers=7) as executor:
        futures = [executor.submit(task) for task in tasks]

if __name__ == "__main__":
    #a = main1()
    #main2(a)
    main3()

1
2
3
4
5
